In [90]:
import pdfminer
import numpy as np
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfinterp
import pdfminer.pdfpage
import re
from mars.utils import extract_text_from_pdf

In [91]:
file_name = 'C:\\Users\jakwi\Downloads\\1906.11668.pdf'

In [108]:
def get_longest(text_list): 
    lengths = [] 
    for text in text_list: 
        lengths.append(len(text))
    lengths = np.array(lengths)
    return text_list[int(np.argmax(lengths))]

def extract_citations_from_jobin2019(filename):
    """Extracts citation numbers from jobin2019 - preprint version"""
    
    # get text
    separated_text = extract_text_from_pdf(file_name)['separated_text']
    
    # getting the longest - the proper text on page
    text_on_pages = [get_longest(st) for st in separated_text]
    text_on_pages = np.array(text_on_pages)
        
    # split words like buckets
    # everything above justice fairness and equity is Transparency 
    # later it is excluded from text and everything above Non-maleficience is Justice, fairness, and equity, etc.
    split_words = [
    'Transparency',
    'Justice, fairness, and equity', 
    'Non-maleficence', 
    'Responsibility and accountability', 
    'Privacy',
    'Beneficence',
    'Freedom and autonomy',
    'Trust',
    'Sustainability',
    'Dignity',
    'Solidarity',
    'Discussion'] # stopword - last one will be excluded 
    
    threads = dict.fromkeys(split_words)
    
    # clean
    for i, text in enumerate(text_on_pages): 
        text = text.replace('\n', '')
        text = text.replace('(cf. Table 2)', '')
        text = text.replace('1.5', '')


        text_on_pages[i] = text
        
    relevant_text = ' '.join(text_on_pages[7:13])

    for i in range(len(list(threads.keys()))-1):
        earlier_thread = list(threads.keys())[i]
        split_thread = list(threads.keys())[i + 1] 
        splitted = relevant_text.split(split_thread, 1)           
        threads[earlier_thread] = splitted[0]

        relevant_text = splitted[1]
        
        # pop empty one
        threads.pop('Discussion')
        
        all_citations = []
        for key, text in threads.items(): 
            citations = np.array(re.findall(r'\d+', text))
            additional_citations = re.findall(r'\d+[–]\d+', text)
            for ac in additional_citations: 
                range_ = re.findall(r'\d+', ac)
                lower, higher = int(range_[0]), int(range_[1])
                between = np.arange(lower+1, higher)
                citations = np.append(citations, between)
            all_citations.append(citations)

        for i, ac in enumerate(all_citations): 
            ac = np.unique(ac)
            all_citations[i] = ac
        
        return all_citations

In [109]:
citations = extract_citations_from_jobin2019(file_name)

justification, domain of application, and mode of achievement. References to transparency comprise efforts to increase explainability, interpretability or other acts of communication and disclosure . Principal domains of application include data use23–26, human-AI interaction23,27–35, automated decisions26,36–46, and the purpose of data use or application of AI systems24,27,47–51. Primarily, transparency is presented as a way to minimize harm and improve  AI36–38,44,45,49,52–55, legal reasons37,45,46,49,50,52  or  to  foster  trust23,24,29,33,36,37,48,51,52,56–58.  A  few  sources  also  link transparency to dialogue, participation, and the principles of democracy30,41,49,50,52,59.  To achieve greater transparency, many sources suggest increased disclosure of information by  those  developing  or  deploying  AI  systems36,51,60,61,  although  specifications  regarding what should be communicated vary greatly: use of AI45, source code31,52,62, data use35,47,50,58, evidence  base  for  A

TypeError: expected string or bytes-like object

[array(['23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
        '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
        '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
        '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66',
        '67', '68', '69', '70', '71'], dtype='<U11'),
 array(['23', '25', '27', '28', '29', '31', '33', '35', '36', '37', '38',
        '39', '41', '42', '43', '44', '45', '46', '47', '48', '50', '51',
        '52', '54', '55', '56', '57', '58', '59', '60', '62', '63', '65',
        '66', '67', '68', '69', '70', '72', '73', '74', '75', '76', '77',
        '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
        '89', '90', '91', '92', '93', '94', '95'], dtype='<U11'),
 array(['100', '101', '102', '23', '25', '27', '30', '31', '32', '33',
        '34', '35', '36', '37', '38', '39', '40', '44', '47', '48', '50',
        '51', '53', '54', '55', '56', '57', '58', '60', '62', '63', '